In [2]:
import pandas as pd
import numpy  as np
import os
import openai

from transformers import GPT2Tokenizer

openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# models = openai.Model.list()
# for model in models['data']:
#     print(model['id'])


# Create Essays with the Same Realtive Entropy of Coherence for their sentences

In [5]:
# Initialize the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [25]:
#FUNCTIONS

# Function to compute the density matrix for text
def text_to_density_matrix(text, tokenizer):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    vector = np.array(tokens, dtype=np.float32) / np.linalg.norm(tokens)
    return vector , np.outer(vector, vector)

# Function to compute von Neumann entropy
def von_neumann_entropy(rho):
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = eigenvalues[eigenvalues > 0]  # Avoid log(0)
    return -np.sum(eigenvalues * np.log2(eigenvalues))

# Function to compute relative entropy of coherence
def relative_entropy_of_coherence(text, tokenizer, token_size):
    rho = text_to_density_matrix(text, tokenizer)[1]
    rho_diag = np.diag(np.diag(rho))
    s_rho =von_neumann_entropy(rho_diag) - von_neumann_entropy(rho)
    return s_rho/np.log(token_size)




# Function to generate a topic
def generate_topic(example_topics, model_type, word_count=15):
    try:
        prompt = (
            f"Consider these example topics: '{example_topics}'. "
            f"Suggest a topic (less than {word_count} words) like the given example but totally different in context and area."
        )

        response = openai.ChatCompletion.create(
            model=model_type,
            messages=[
                {"role": "system", "content": "You are an expert in suggesting topics in various fields."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=word_count * 4 // 3,
            temperature=0.7
        )

        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating topic: {str(e)}"







# Function to generate an essay for a topic
def generate_essay(topic, model_type, sentence_count=5, max_words=20):
    
    sentences = []
    while len(sentences) < sentence_count:
        prompt = (
            f"Write a single sentence related to the topic '{topic}' with unique phrasing."
            f"The sentence should be around '{max_words}' words."
            f"Put '.' at the end of each sentence."
        )
        

        response = openai.ChatCompletion.create(
            model=model_type,
            messages=[
                {"role": "system", "content": "You are an expert in writing unique sentences."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_words * 4 // 3,
            temperature=0.7
        )

        sentence = response.choices[0].message.content.strip()
        token_size = len(tokenizer.encode(sentence))
        coherence = relative_entropy_of_coherence(sentence, tokenizer, token_size)
        
        coherence_range = [0.75 *coherence , 1.25 * coherence] 
        print(sentence)
        print(coherence)
        # aa = input("Press Enter")

        if coherence_range[0] <= coherence <= coherence_range[1] and sentence not in sentences:
            sentence = sentence.replace('"', '').replace("'", "")
            sentences.append(sentence)
        print("="*80)

    return sentences


In [8]:
topics = [
    "Impact of AI on Climate Change",
    "Philosophical Implications of Quantum Computing",
    "Advancements in Medical Robotics"
]

# topics = [generate_topic(example_topics, model_type) for _ in range(3)]

for _ in range(3):
    new_topic = generate_topic(topics, "gpt-4")
    new_topic = new_topic.replace('"', '')
    topics.append(new_topic)

topics

['Impact of AI on Climate Change',
 'Philosophical Implications of Quantum Computing',
 'Advancements in Medical Robotics',
 'Evolution of Space Tourism and its Economic Implications',
 'Effects of Social Media on Mental Health in Adolescents',
 'Exploration of Deep Sea Biodiversity and its Potential Biotechnological Applications']

In [28]:
df = pd.read_csv("topics.csv")

essay_list = []
for topic in df["topic"]:
    print(topic)
    essay_sentences = generate_essay(topic, "gpt-4")
    essay = ''
    for item in essay_sentences:
        if item[-1] != ".":
            item += "."
        essay += " " + item
    essay_list.append(essay)

df["essay"] = essay_list

df.to_csv("essay2.csv", index=False)    

Compare and contrast the importance of self-reliance and adaptability in healthcare.
In healthcare, self-reliance is key for professional growth, but adaptability is pivotal in responding to ever-evolving medical situations
0.6622299622733891
In healthcare, self-reliance empowers individual decision-making, whereas adaptability fosters responsiveness to evolving patient needs and medical advancements
0.7332154624831863
In healthcare, the virtue of self-reliance empowers personal growth, while adaptability equips practitioners for evolving medical landscapes.
0.7708760995331083
In healthcare, self-reliance empowers personal health autonomy, whereas adaptability fosters versatility in ever-evolving medical landscapes.
0.8329329434109455
In healthcare, self-reliance empowers personal health management, while adaptability navigates dynamic medical landscapes, each indispensable in their
0.7228701181168311
Evaluate the effectiveness of management consulting in addressing conflicts within ma

In [ ]:
topic = topics[0]
print(f"\nTopic: {topic}\n")
essay_sentences = generate_essay(topic, "gpt-4")

# # Step 2: Generate essays for each topic
# for topic in topics:
#     print(f"\nTopic: {topic}\n")
#     essay_sentences = generate_essay(topic, model_type)
#     for sentence in essay_sentences:
#         print(sentence)




Topic: Impact of AI on Climate Change

"AI's potential to streamline renewable energy use could orchestrate a symphony of sustainability, mitigating climate change's abrasive crescendo
0.8074849576193053
Amir
Harnessing the prowess of artificial intelligence could be our key to deciphering and mitigating the cryptic puzzles of climate change.
0.7585043032319406
Amir
Harnessing AI's predictive prowess could be our secret weapon in the battle against climate change, optimizing our response strategies.
0.6410876329664223
Amir
Harnessing the power of artificial intelligence, we can potentially revolutionize climate change mitigation strategies with precise predictions and innovative solutions.
0.7817798176769343
Amir
Harnessing AI's analytical prowess could offer groundbreaking solutions, transforming our battle against the menacing specter of climate change.
0.8338089066440018
Amir


'AIs potential to streamline renewable energy use could orchestrate a symphony of sustainability, mitigating climate changes abrasive crescendoHarnessing the prowess of artificial intelligence could be our key to deciphering and mitigating the cryptic puzzles of climate change.Harnessing AIs predictive prowess could be our secret weapon in the battle against climate change, optimizing our response strategies.Harnessing the power of artificial intelligence, we can potentially revolutionize climate change mitigation strategies with precise predictions and innovative solutions.Harnessing AIs analytical prowess could offer groundbreaking solutions, transforming our battle against the menacing specter of climate change.'

In [24]:
essay = ''
for item in essay_sentences:
    if item[-1] != ".":
        item += "."
    essay += " " + item

essay

' AIs potential to streamline renewable energy use could orchestrate a symphony of sustainability, mitigating climate changes abrasive crescendo. Harnessing the prowess of artificial intelligence could be our key to deciphering and mitigating the cryptic puzzles of climate change. Harnessing AIs predictive prowess could be our secret weapon in the battle against climate change, optimizing our response strategies. Harnessing the power of artificial intelligence, we can potentially revolutionize climate change mitigation strategies with precise predictions and innovative solutions. Harnessing AIs analytical prowess could offer groundbreaking solutions, transforming our battle against the menacing specter of climate change.'

In [29]:

from transformers import GPT2Tokenizer

# Initialize the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


def relative_entropy_of_coherence(text, tokenizer, token_size):
    rho = text_to_density_matrix(text, tokenizer)[1]
    rho_diag = np.diag(np.diag(rho))
    s_rho =von_neumann_entropy(rho_diag) - von_neumann_entropy(rho)
    return s_rho/np.log(token_size)


In [32]:
text= "Harnessing the computational prowess of artificial intelligence, humans have initiated a technological crusade against the escalating menace of climate change."

tokens = tokenizer.encode(text)


token_size = len(tokens)
print(token_size)

relative_entropy_of_coherence(text , tokenizer, token_size)

24


np.float64(0.9006643354822662)

In [26]:
a = text_to_density_matrix(text, tokenizer)[0].T
np.dot(a, text_to_density_matrix(text, tokenizer)[0])

np.float64(1.0000000000000002)